In [ ]:
!pip install .

In [2]:
from msdis_lidar.shapefile_handler import ShapefileHandler
from msdis_lidar.downloader import LidarDownloader

In [3]:
import msdis_lidar

In [ ]:
help(LidarDownloader)

In [4]:
handler = ShapefileHandler("Perche_Creek_HU12.shp")

In [5]:
polys = handler.get_polygon_string()

10 geometries found

CRS = EPSG:3857 is not compatible

Projecting to EPSG:4326 ...

Simplifying the polygons...

Output()

In [6]:
downloader = LidarDownloader("ddd")

In [7]:
url_gdf = downloader.fetch_lidar_download_links(polys)

Output()

AttributeError: 'GeoDataFrame' object has no attribute 'append'

In [ ]:
url_gdf

In [8]:
def get_tile_info(info_gdf, i):
    # i is the tile index in the LiDAR infor geodatabase 
    # this geodatbase is the output of running fetch_lidar_download_links tool for an input polygon
    name =  info_gdf.iloc[i]['ftp_1'].split('/')[-1]
    url =  info_gdf.iloc[i]['updated_ft']
    
    return name, url    

In [9]:
from rich.progress import track, Progress, BarColumn, TextColumn
from rich.console import Console
import requests
import zipfile
import shutil
import time
from rich import print
import os

In [10]:
name, url = get_tile_info(url_gdf, 78)
print(name, url)

NameError: name 'url_gdf' is not defined

In [ ]:
download_file(url, name)

In [ ]:
def download_file(url, name):
    local_filename = name
    
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename

In [ ]:
def unzip_file(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()

In [ ]:
def main(url, name):   
    # Step 1: Download the file
    zip_path = download_file(url, name)
    # Step 2: Check if the downloaded file is a zip file
    if zipfile.is_zipfile(name):
        # Step 3: Unzip the file
#         print(f'unzipping the file...', end = '\r')
        unzip_file(name)
        
        # Step 4: Delete the original zip file
        os.remove(name)
        
        las_file = name[:-4]

    else:
#         print(f"{zip_path} is not a zip file, proceesing with las file name", end = '\r')
        las_file = name
        
    return(las_file)

### Minimum Sampling Lines

In [ ]:
def get_las_info(las_file):
    las = laspy.read(las_file)
    
    # extracting the ground points from las file
    classifications = las.classification
    ground_points = las.points[classifications==2]
    x = ground_points.x
    y = ground_points.y
    z = ground_points.z  
    
    # Define the bounding box for random centroid locations (adjust as needed)
    min_x, max_x = np.min(x),np.max(x)
    min_y, max_y = np.min(y),np.max(y)
    
    A = (max_x - min_x ) * (max_y - min_y)
    return x, y, z, min_x, max_x, min_y, max_y, A

In [ ]:
x, y, z, min_x, max_x, min_y, max_y, A = get_las_info(las_file)

In [ ]:
def min_samples(A, L1 =300, L2 = 1000, Pc = 0.95):
    # A = area of the las tile in square feet
    # Pc = confidence level from 0 to 1.
    P0 = 1-(2*L1*L2/(np.pi*A))
    print(P0)
    if P0<0:
        num_of_lines = int(5.0)
    else:
        num_of_lines = int(np.ceil(np.log(1-Pc)/np.log(P0)))
    
    print(f' {num_of_lines} sampling line is needed for {Pc*100}% confidence level')
    return num_of_lines

In [ ]:
min_samples(A, L2=1000, Pc=0.95)

In [ ]:
import numpy as np
import laspy
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import random

In [ ]:
def get_sample_lines(num_of_lines, min_x, max_x, min_y, max_y, line_length = 300):
    
    # Create an empty GeoDataFrame to store the random lines
    gdf = gpd.GeoDataFrame(columns=['geometry'])


    # Generate random lines
    for _ in range(num_of_lines):
        centroid_x = random.uniform(min_x, max_x)
        centroid_y = random.uniform(min_y, max_y)
        
        orientation_angle_degrees = random.uniform(0, 360)

        orientation_angle_radians = orientation_angle_degrees * (3.14159265359 / 180.0)

        line_endpoint_x1 = centroid_x - (line_length/2)*np.cos(orientation_angle_radians)
        line_endpoint_y1 = centroid_y - (line_length/2)*np.sin(orientation_angle_radians)
        line_endpoint_x2 = centroid_x + (line_length/2)*np.cos(orientation_angle_radians)
        line_endpoint_y2 = centroid_y + (line_length/2)*np.sin(orientation_angle_radians)

        line = LineString([(line_endpoint_x1, line_endpoint_y1), (line_endpoint_x2, line_endpoint_y2)])

        gdf = gdf.append({'geometry': line}, ignore_index=True)
        
    return gdf

In [ ]:
# gdf_lidar = gpd.GeoDataFrame(columns=['geometry', 'elv_data'])   
    
# for tile_no in range(len(url_gdf)):
#     name, url = get_tile_info (url_gdf, tile_no)
    
#     print(f'\rdownloading tile # {tile_no} out of {len(url_gdf)}...', end = '', flush = True)
#     las_file = main(url, name)
#     print(f'\r             {las_file} downloaded successfully.', end = '')

#     #getting las file info
#     print(f'\rgetting las file information for LiDAR tile {las_file}...', end = '')
#     x, y, z, min_x, max_x, min_y, max_y, A = get_las_info(las_file)
#     print(f'\r                      information extracted susscessfully for tile {las_file}.', end = '')

    
#     # how many sampling line we might nee
#     num_of_lines = min_samples(A, L2 = 1250, Pc = 0.95)
    
    
#     # extracting sampling lines
#     gdf = get_sample_lines(num_of_lines, min_x, max_x, min_y, max_y, line_length = 300)
    
#     print(f'\rextracting cross section elevation data for sample lines...', end = '')
#     for i in range(num_of_lines):
#         cs_data, cs_line = get_cross_section_data(gdf, i, x, y, z)
#         Delta_ELV = np.max(cs_data.T[2])-np.min(cs_data.T[2])
#         if Delta_ELV > 2.0:
#             gdf_lidar = gdf_lidar.append({'geometry': cs_line, 'elv_data':cs_data}, ignore_index=True)
        
#     print(f'\r                      elevation data extracted successfully for tile {tile_no+1}/{len(url_gdf)}', end='')

    
#     # Delete the original las file
#     os.remove(las_file)
#     print(f'\r{las_file} removed.', end = '')
#     print(f'\r--------------------------------------------------------------------------------------------', end = '')

In [ ]:
help(progress.add_task)

In [ ]:
help(progress.update)

In [ ]:
from rich.progress import Progress, track
from rich.console import Console
import geopandas as gpd
import os

# Initialize console for rich logging
console = Console()

# Initialize an empty GeoDataFrame
# gdf_lidar = gpd.GeoDataFrame(columns=['geometry', 'elv_data'])

# Using rich Progress for better progress tracking
with Progress(
    TextColumn("[progress.description]{task.description}"),
    BarColumn(),
    TextColumn("[progress.percentage]{task.percentage:>3.2f}%"),  # Default precision
    console=console
    
) as progress:
    task = progress.add_task("Downloading tiles...", total=len(url_gdf)-768)

    for tile_no in range(1171,len(url_gdf)):
        name, url = get_tile_info(url_gdf, tile_no)
        progress.update(task, description=f"Downloading tile # {tile_no + 1} of {len(url_gdf)}", advance=1)
        las_file = main(url, name)
        console.log(f"[bold green]{las_file} downloaded successfully.[/bold green]")

        # Getting las file info
        console.log(f"Getting las file information for LiDAR tile {las_file}...")
        x, y, z, min_x, max_x, min_y, max_y, A = get_las_info(las_file)
        console.log(f"[bold green]Information extracted successfully for tile {las_file}.[/bold green]")

        # Determine how many sampling lines are needed
        num_of_lines = min_samples(A, L2=1000, Pc=0.95)
        
        # Extracting sampling lines
        console.log("Extracting cross section elevation data for sample lines...")
        gdf = get_sample_lines(num_of_lines, min_x, max_x, min_y, max_y, line_length=300)
        
        for i in range(num_of_lines):
            cs_data, cs_line = get_cross_section_data(gdf, i, x, y, z)
            if cs_data.size ==0: 
                print("nothing to add, no data here.")
            else:
                Delta_ELV = np.max(cs_data.T[2])-np.min(cs_data.T[2])
                if Delta_ELV > 2.0:
                    gdf_lidar = gdf_lidar.append({'geometry': cs_line, 'elv_data': cs_data}, ignore_index=True)
                else:
                    print("nothing to add, too flat of a place")
        
        console.log(f"[bold green]Elevation data extracted successfully for tile {tile_no + 1}/{len(url_gdf)}.[/bold green]")

        # Delete the original las file
        os.remove(las_file)
        console.log(f"[red]{las_file} removed.[/red]")


In [ ]:
las_file

In [ ]:
gdf_lidar

In [ ]:
memory_usage = gdf_lidar.memory_usage(deep=True)
print(memory_usage)

total_memory = memory_usage.sum()
print(f"Total memory usage: {total_memory / (1024 ** 2):.2f} MB")

In [ ]:
gdf_lidar

In [ ]:
gdf = get_sample_lines(num_of_lines, min_x, max_x, min_y, max_y)
print(gdf)

In [ ]:
def get_cross_section_data(gdf, i, x, y, z):
       
    cross_section_line = gdf["geometry"][i]
    # line start and end point 
    x1 = cross_section_line.coords[0][0]
    y1 = cross_section_line.coords[0][1]
    x2 = cross_section_line.coords[1][0]
    y2 = cross_section_line.coords[1][1]


    # having three points on a plane we can define the plane
    # as we have 2 points in 2D space we assume a third point in 
    # an arbitray elevation z = 5. This will enforce a vertical plane 
    # passes throght our line. 
    P0 = np.array([x1, y1, 0])
    P1 = np.array([x1, y1, 5])
    P2 = np.array([x2, y2, 0])

    # We can find its normal vector by applying cross product rule. 
    N = np.cross(P1-P0, P2-P1)
    unit_N = N/np.linalg.norm(N)  # unit normal vector 

    # stacking all the points from the LAS file 
    points = np.vstack([x, y, z]).T

    # choosing on point on our sampling line
    plane_point = np.array([x2, y2, 0.0])  # Example point on the plane

    # vector from each elevation point to starting point on the sampling line. 
    V = points - plane_point

    # The distance between each elevation point and the verticl plane passing from our sampling line 
    distance = np.dot(V, unit_N)

    # to what distance of sampling plane we want to get elevation points 
    thickness = 6.0  # Cross section thickness


    # Extract cross section points
    cross_section_points = points[np.abs(distance) < thickness/2]


    # this section filters points that are between the start and end point of the sampling line
    def is_between(point, P2, P0 ):
        D = P2- P0  # direction of normal vector is from P2 to P0
        unit_D = D/np.linalg.norm(D)
        V2 = point - P2    # distance to point on start plane
        V0 = point - P0    # distance to point on end plane

        d2 = np.dot(V2, unit_D)  # prependicular distance to start plane
        d0 = np.dot(V0, unit_D)  # prependicular distance to end plane

        return(d0>=0 and d2<=0)

    #using the filter we defined above
    cross_section_points = np.array([point for point in cross_section_points if is_between(point, P2, P0)])    
    return cross_section_points, cross_section_line

In [ ]:
gdf_lidar = gpd.GeoDataFrame(columns=['geometry', 'elv_data'])

In [ ]:
gdf_lidar

In [ ]:
# for i in all_the_tiles:
#     create an empty geodataframe to store the sampling line and thier elevation sequence and the centroid of each line 
#     download i
#     if i is zip
#     unzip i 
#     get the area
#     get the crs
#     calcualte the number of sampling lines
#     generating randomly located and randomly oriented lines 
#     extract the elevation data for each sampling line to the geodataframe created earlier
    
    

In [ ]:
# saviong the data
gdf_lidar.to_csv("LiDAR_data_Perche_Creek.csv", index=False)